In [20]:
import pandas as pd
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from tqdm import tqdm
import sys
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from ResUNet_model import ResUNet


In [21]:
# Load metadata
df_metadata = pd.read_csv("../data/segmentation/sample_archive/metadata.csv")

# Split df_metadata into train and test by the "split" column
train_df = df_metadata.loc[df_metadata['split'] == 'train']
test_df = df_metadata.loc[df_metadata['split'] == 'test']

train_df

,image_id,split,sat_image_path,mask_path
0,992507,train,train/992507_sat.jpg,train/992507_mask.png
1,994520,train,train/994520_sat.jpg,train/994520_mask.png
2,995492,train,train/995492_sat.jpg,train/995492_mask.png
3,997521,train,train/997521_sat.jpg,train/997521_mask.png
4,998002,train,train/998002_sat.jpg,train/998002_mask.png


In [22]:
# Load classes
df_classes = pd.read_csv("../data/segmentation/sample_archive/class_dict.csv")
df_classes

,name,r,g,b
0,urban_land,0,255,255
1,agriculture_land,255,255,0
2,rangeland,255,0,255
3,forest_land,0,255,0
4,water,0,0,255
5,barren_land,255,255,255
6,unknown,0,0,0


In [23]:
import cv2
import numpy as np

def preprocess_data(data, image_size=(128, 128)):
    X = []
    y = []

    for index, row in data.iterrows():
        # Load image
        image_path = "../data/segmentation/sample_archive/" + row['sat_image_path']
        image = cv2.imread(image_path)
        # Resize image if needed
        image = cv2.resize(image, image_size)
        # Normalize image
        image = image / 255.0

        # Load mask
        mask_path = "../data/segmentation/sample_archive/" + row['mask_path']
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        # Resize mask if needed
        mask = cv2.resize(mask, image_size)
        # Normalize mask
        mask = mask / 255.0

        X.append(image)
        y.append(mask)

    return np.array(X), np.array(y)

# Prepare dataset for training
X, y = preprocess_data(train_df)




In [24]:
# Split train dataset into further labelled train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)



(4, 128, 128, 3) (1, 128, 128, 3) (4, 128, 128) (1, 128, 128)


In [25]:
# Flatten the images and masks
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)
y_train_flat = y_train.reshape(y_train.shape[0], -1)
y_test_flat = y_test.reshape(y_test.shape[0], -1)

In [26]:
# Define ResUNet model
model = ResUNet(input_shape=(128, 128, 3), num_classes=7)

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4),
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_test, y_test))

# Evaluate the model
accuracy = history.history['val_accuracy'][-1]
print("Validation Accuracy:", accuracy)

ModuleNotFoundError: No module named 'tensorflow'